In [1]:
import itertools
import os
import numpy as np
import pandas as pd

In [2]:
d_experiment_logs = './logs/'

In [3]:
test_logs = [f"{d_experiment_logs}/{x}" for x in sorted(os.listdir(d_experiment_logs)) if x.startswith('test')]
train_logs = [f"{d_experiment_logs}/{x}" for x in sorted(os.listdir(d_experiment_logs)) if x.startswith('train')]
test_logs, train_logs

(['./logs//test_1.log',
  './logs//test_2.log',
  './logs//test_3.log',
  './logs//test_4.log',
  './logs//test_5.log',
  './logs//test_6.log',
  './logs//test_7.log',
  './logs//test_8.log'],
 ['./logs//train_1.log',
  './logs//train_2.log',
  './logs//train_3.log',
  './logs//train_4.log',
  './logs//train_5.log',
  './logs//train_6.log',
  './logs//train_7.log',
  './logs//train_8.log'])

In [4]:
def add_metric_val(metrics: dict, dataset: str, method: str, metric_name: str, metric_val: float) -> None:
    method = method.split('-')[0]
    if dataset not in metrics:
        metrics[dataset] = {}
    if method not in metrics[dataset]:
        metrics[dataset][method] = {}
    if metric_name not in metrics[dataset][method]:
        metrics[dataset][method][metric_name] = []
    metrics[dataset][method][metric_name].append(metric_val)


total_dfs = dict()

for train_log, test_log in zip(train_logs, test_logs):
    train_params = None
    metrics = {}
    with open(train_log) as train_in, open(test_log) as test_in:
        for line in test_in:
            if line.startswith('Namespace('):
                line = line.replace('Namespace', 'dict')
                train_params = eval(line)
            else:
                if line.startswith(' * Acc@1'):
                    acc = float(line.split()[-1])
                    add_metric_val(metrics, train_params['data_name'], train_params['log'], 'accuracy', acc)
                elif line.startswith(' * Acc1'):
                    acc = float(line.split()[2])
                    add_metric_val(metrics, train_params['data_name'], train_params['log'], 'accuracy', acc)
                elif line.startswith('PR AUC') or line.startswith('F1 PR AUC'):
                    pr_auc = float(line.split()[-1])
                    add_metric_val(metrics, train_params['data_name'], train_params['log'], 'pr_auc', pr_auc)
                elif line.startswith('ROC AUC') or line.startswith('F1 ROC AUC'):
                    roc_auc = float(line.split()[-1])
                    add_metric_val(metrics, train_params['data_name'], train_params['log'], 'roc_auc', roc_auc)
    
        for ds_name, ds in metrics.items():
            cur_metrics = None
            result = []
            for method_name, method_data in ds.items():
                if cur_metrics is None:
                    cur_metrics = list(method_data.keys())
                row = dict(zip(['method', *cur_metrics],
                               [method_name] +
                               [f'{np.mean(method_data[metric]):.3f}' for metric in cur_metrics]))
                if method_name == 'src_only':
                    result.insert(0, row)
                else:
                    result.append(row)
            df = pd.DataFrame(result).set_index('method').astype(float)

            if ds_name not in total_dfs.keys():
                total_dfs[ds_name] = df.sort_index()
            else:
                total_dfs[ds_name] = pd.concat([total_dfs[ds_name], df]).sort_index()

import functools
resulting_df = functools.reduce(lambda x, y: pd.concat([x, y], axis=1), total_dfs.values())
resulting_df.columns = pd.MultiIndex.from_product([[k for k in total_dfs.keys()], list(total_dfs.values())[0].columns], names=['data', 'metric_type'])
display(resulting_df)

data        His.ALL.05.H3K79me1.AllCell.dm6.mm10                  \
metric_type                             accuracy  pr_auc roc_auc   
method                                                             
adda                                      72.064  80.279  80.571   
afn                                       76.649  87.276  85.090   
cdan                                      67.753  74.619  76.440   
dan                                       70.620  78.989  81.306   
dann                                      67.342  74.567  77.923   
jan                                       66.458  71.571  75.149   
mcc                                       66.320  74.205  74.552   
mcd                                       78.057  87.813  86.890   
mdd                                       66.148  74.360  72.414   
src_only                                  75.684  84.573  84.069   

data        His.ALL.05.H3K79me1.AllCell.hg38.ce11                  \
metric_type                              accuracy  pr_auc roc_auc   
method                                                              
adda                                       51.136  61.861  55.846   
afn                                        56.676  61.314  56.082   
cdan                                       53.125  55.542  60.980   
dan                                        43.466  48.993  40.648   
dann                                       50.000  49.948  49.961   
jan                                        46.449  50.905  46.630   
mcc                                        47.953  50.376  42.016   
mcd                                        50.426  55.938  44.054   
mdd                                        57.244  57.201  59.631   
src_only                                   59.801  62.410  55.869   

data        His.ALL.05.H3K79me1.AllCell.ce11.mm10                  \
metric_type                              accuracy  pr_auc roc_auc   
method                                                              
adda                                       57.091  62.195  64.091   
afn                                        68.191  78.216  78.335   
cdan                                       68.444  79.944  78.894   
dan                                        64.796  73.241  75.901   
dann                                       68.030  79.929  78.784   
jan                                        66.413  80.020  78.362   
mcc                                        69.432  80.009  79.269   
mcd                                        62.351  73.019  76.963   
mdd                                        65.287  73.219  76.922   
src_only                                   66.015  71.292  75.143   

data        His.ALL.05.H3K79me1.AllCell.mm10.hg38  ...  \
metric_type                              accuracy  ...   
method                                             ...   
adda                                       53.693  ...   
afn                                        55.966  ...   
cdan                                       62.216  ...   
dan                                        57.670  ...   
dann                                       57.102  ...   
jan                                        52.841  ...   
mcc                                        63.889  ...   
mcd                                        55.966  ...   
mdd                                        57.955  ...   
src_only                                   61.080  ...   

data        His.ALL.05.H3K79me1.AllCell.mm10.ce11  \
metric_type                               roc_auc   
method                                              
adda                                       57.921   
afn                                        39.119   
cdan                                       50.538   
dan                                        45.330   
dann                                       45.272   
jan                                        46.013   
mcc                                        44.294   
mcd                               

In [6]:
resulting_df.loc[:, resulting_df.columns.get_level_values(0).str.contains('mm10.hg38|hg38.mm10')]

data        His.ALL.05.H3K79me1.AllCell.mm10.hg38                  \
metric_type                              accuracy  pr_auc roc_auc   
method                                                              
adda                                       53.693  54.429  54.667   
afn                                        55.966  54.956  52.150   
cdan                                       62.216  60.335  60.802   
dan                                        57.670  58.349  57.568   
dann                                       57.102  55.563  55.531   
jan                                        52.841  58.290  57.403   
mcc                                        63.889  64.291  63.264   
mcd                                        55.966  60.269  58.065   
mdd                                        57.955  56.252  55.068   
src_only                                   61.080  63.960  62.240   

data        His.ALL.05.H3K79me1.AllCell.hg38.mm10                  
metric_type                              accuracy  pr_auc roc_auc  
method                                                             
adda                                       67.958  78.540  80.290  
afn                                        72.523  79.769  78.661  
cdan                                       63.674  69.564  69.360  
dan                                        69.719  72.068  75.466  
dann                                       62.009  65.869  66.634  
jan                                        62.769  64.356  67.152  
mcc                                        66.808  74.132  72.744  
mcd                                        66.695  71.375  73.396  
mdd                                        68.376  73.156  75.076  
src_only                                   67.845  78.722  76.738

In [201]:
import plotly.express as px

plotly_df = pd.DataFrame()
for data_name, data_df in total_dfs.items():
    reference = data_df.loc["src_only", :].to_frame().T
    reference.index.name = "method"

    diff_df = (
        pd.DataFrame(
            np.subtract(data_df.values, reference.values),
            index=data_df.index,
            columns=data_df.columns,
        ).iloc[:-1,:]
        .reset_index()
        .assign(data_name="->".join(data_name.rsplit(".", maxsplit=2)[-2:]))
        .melt(
            id_vars=["data_name", "method"],
            value_vars=["accuracy", "pr_auc", "roc_auc"],
            var_name='metric',
            value_name='diff'
        )
    )

    plotly_df = pd.concat([plotly_df, diff_df])

plotly_df = plotly_df.sort_values(['data_name', 'method','metric'], ascending=[False, True, True])
plotly_df


,data_name,method,metric,diff
0,mm10->hg38,adda,accuracy,-7.387
9,mm10->hg38,adda,pr_auc,-9.531
18,mm10->hg38,adda,roc_auc,-7.573
1,mm10->hg38,afn,accuracy,-5.114
10,mm10->hg38,afn,pr_auc,-9.004
...,...,...,...,...
16,ce11->dm6,mcd,pr_auc,2.110
25,ce11->dm6,mcd,roc_auc,0.794
8,ce11->dm6,mdd,accuracy,3.350
17,ce11->dm6,mdd,pr_auc,3.702


In [220]:
fig = px.bar(
    plotly_df,
    facet_col="data_name",
    x="metric",
    y="diff",
    color="method",
    barmode="group",
)
fig.update_layout(
    title=f'{list(total_dfs.keys())[0].rsplit(".", maxsplit=2)[0]}',
    height=400,
    width=1800,
    margin=dict(l=10, r=10, t=50, b=10),
)

fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.for_each_xaxis(lambda y: y.update(title=""))
fig.add_annotation(
    y=-0.25,
    x=0.5,
    text="metric",
    xref="paper",
    yref="paper",
    showarrow=False,
)

path_output = f'{dir_output}/{list(total_dfs.keys())[0].rsplit(".", maxsplit=2)[0]}'
fig.write_html(f"{path_output}.html")
fig.write_image(f"{path_output}.png")